# Hybrid Approximate Nearest Neighbors Algorithm for Movie Recommendations

## By Cody Zupnick, Kathy Lin and Priya Medberry

Our objective will be to maximize the accuracy of the recommendations provided to Movielens users when considering the results of recommending the Top-5 movies for a given user. For this purpose, we will be using a dataset from Movielens with over 10 million ratings. This notebook contains 5 sections: 
1.  Business Objectives/Background
2.  Exploratory Data Analysis
3.  Motivation for Hybrid Model
4.  Implementation of Hybrid Model
5.  Performance of Hybrid Model


Firstly, we'll provide some background on the business objectives of Movielens. Then we'll dive into the dataset and conduct some exploratory data analysis. Then we will describe the motivation and implementation of a hybrid model used to produce these recommendations. Lastly, we will compare the performance of our designed hybrid model against the performance of prior models such as a raw item-based collaborative filtering model, and the matrix factorization model. 

### 1. Business Objectives/Background
 MovieLens is a web-based recommender system and virtual community that recommends movies for its users to watch, based on their film preferences using CF of members' movie ratings and reviews. As with any recommendation system, the business objective is to build trust between the consumer and the recommendation system by providing accurate predictions of items that a consumer may like based on history of user interactions with the system (both implicitly and explicitly). For the purposes of this dataset, we'll be focusing solely on explicit ratings provided in the past by the users. In our case, people tend to like movies that are similar to other movies they like, and they tend to have similar taste to others they are close with. So as a business, we will try to capture these behavioral patterns to help predict what movies an individual hasn't watched and help expose those users to potential movies we think they'd like.
 
 However, it is important to note early on that there are often 3 types of common challenges with implementing these algorithms for the purpose of provide accurate recommendations: time, scale and quality. Although it is nice to produce accurate recommendations in a short period of time given large numbers of users and movies, there is a tradeoff between computation time and the number of movies / users in the system. Likewise, the faster an algorithm runs, which is also known as more scalability, the worse quality recommendations it usually provides. Therefore, we have designed in this project a potential solution to this tradeoff triangle. We'd still like to maximize accuracy in a reasonable period of time for a large number of users and movies. Therefore, we will use a hybrid model of approximate nearest neighbors method (item-based collaborative filtering method) with a dimensionality reducing algorithm. This hybrid model will be able to counteract the problem of scalability.


 ### 2. Exploratory Data Analysis

We start by trying to understand the Movielens 10M dataset a bit better. There are two main datasets we will be working with: **Movies.dat** and **Ratings.dat**


** 1. Movies.dat**

Each line of this file represents one movie, and has the following format: MovieID::Title::Genres
MovieID is the real MovieLens id.

Movie titles, by policy, should be entered identically to those found in IMDB, including year of release. However, they are entered manually, so errors and inconsistencies may exist.Genres are a pipe-separated list, and are selected from the following:

Action
Adventure
Animation
Children's
Comedy
Crime
Documentary
Drama
Fantasy
Film-Noir
Horror
Musical
Mystery
Romance
Sci-Fi
Thriller
War
Western


** 2. Ratings.dat **

All ratings are contained in the file ratings.dat. Each line of this file represents one rating of one movie by one user, and has the following format:

UserID::MovieID::Rating::Timestamp

The lines within this file are ordered first by UserID, then, within user, by MovieID. Ratings are made on a 1-5 star scale, with half-star increments. Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

We read in these datasets using the Pandas library:

In [3]:
import pandas as pd

moviescol = ['MovieId', 'Title', 'Genres','Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

ratings = pd.read_csv('./ratings.dat', sep = '::', names = ['UserId', 'MovieId', 'Rating', 'Timestamp'], engine = 'python')
movies = pd.read_csv('./movies.dat', sep ='::', names = moviescol, engine='python')

### 3. Motivation for Hybrid Model
One of the simplest approaches we could take to recommend the top 5 movies, is to recommend the movies liked by the most number of users; that is, the most popular items. Although this is a very efficient approach, it has a major drawback -- there is absolutely no personalization involved. Therefore, we have chosen not to move forward with this approach since the most popular items would be the same for each user, and everyone would see the same results. We felt that this methodology would not promote trust between our movie recommendation system and consumers.

Next, we consider item-based collaborative filtering (approximate nearest neighbors method). Intuitively, we had mentioned earlier that people tend to like similar items to users who behaved similarly in the past. Item-based collaborative filtering will take a movie, find users who liked that movie, and find other movies that those users or similar users have liked. In otherwords, we compute the similarity between users and items and consider the closest items and users in these "neighborhoods". Therefore, we can provide our consumers a transparent reason behind their recommendations that is similar to "Users who have liked this movie also liked..."

However, there are two major drawbacks of this method as well that we've discovered previously. Firstly, it doesn't scale particularly well with massive datasets; therefore, we're wary of proceeding with purely an approximate nearest neighbors approach with over 10 million ratings in our dataset. Next, there's a concern about quality of recommendations. Given our exploratory analysis shows that there is a high level of sparsity in our dataset, we may be computing similarity between items and users purely on noise inherent in the data. In other words, we may be constructing neighbors matched on sparse, low-level details that we assume represent the user's preference vectors instead of matching on the vectors inherent in the data itself. For example, if one user has watched the first four Harry Potter movies and another watched the last four, the raw user item matrix constructed in this approximate nearest neighbors method wouldn't have any overlap between these two users. Therefore, they'd lie in separate neighborhoods, even though it is clear that these two users share some underlying preferneces. 

Therefore, we'd like to build a hybrid model that combines another method with the approximate nearest neighbors method to overcome this apparent issue. This method would derive both tastes and preferences from the data using matrix factorization. We'll describe the implementation of our hybird model in the next section.

### 4. Implementation of the Hybrid Model

### 5. Performance of the Hybrid Model